In [36]:
from Bio import SeqIO

def extractSequences(filename, outfile):
    fasta_sequences = SeqIO.parse(open(filename),'fasta')
    sequences = []
    ids = []

    out = open(outfile, "w")

    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        ids.append(name)
        sequences.append(fasta)

        seq_len = len(fasta)
        print(name)
        for n in range(2, seq_len + 1):
            for i in range(seq_len - n + 1):
                line = name + " " + sequence[i:i+n] + " " + str(i) + "\n"
                out.write(line)
    out.close()

In [37]:
def writeSequencesToFile(inFiles, outFiles):
    for i in range(len(inFiles)):
        extractSequences(inFiles[i], outFiles[i])

In [40]:
outFiles = ["sequences/sequences2.txt"]
inFiles = ["preprocessedFiles/preprocessed2.fasta"]

writeSequencesToFile(inFiles, outFiles)

DP00004
DP00007
DP00011
DP00016
DP00017
DP00018
DP00028
DP00054
DP00062
DP00069
DP00073
DP00075
DP00079
DP00084
DP00086
DP00092
DP00119
DP00123
DP00134
DP00141
DP00153
DP00162
DP00173
DP00196
DP00214
DP00219
DP00223
DP00230
DP00231
DP00236
DP00238
DP00262
DP00287
DP00307
DP00308
DP00309
DP00311
DP00315
DP00320
DP00322
DP00324
DP00327
DP00332
DP00333
DP00339
DP00343
DP00357
DP00358
DP00364
DP00392
DP00393
DP00420
DP00428
DP00448
DP00466
DP00468
DP00485
DP00486
DP00492
DP00506
DP00517
DP00520
DP00521
DP00537
DP00539
DP00543
DP00546
DP00549
DP00554
DP00555
DP00558
DP00565
DP00570
DP00581
DP00607
DP00608
DP00611
DP00621
DP00630
DP00633
DP00641
DP00652
DP00670
DP00672
DP00683
DP00694
DP00704
DP00706
DP00709
DP00716
DP00718
DP00736
DP00747
DP00748
DP00757
DP00793
DP00796
DP00816
DP00841
DP00846
DP00864
DP00865
DP00873
DP00893
DP00901
DP00903
DP00910
DP00912
DP00917
DP00934
DP00937
DP00941
DP00949
DP00951
DP00959
DP00961
DP00962
DP00969
DP01054
DP01063
DP01065
DP01098
DP01099
DP01100
DP01102


OSError: [Errno 28] No space left on device

In [2]:
def findNgrams(sequence):
    n = len(sequence)
    ngrams = []
    for k in range(2, n):
        for i in range(n - k + 1):
            igram = sequence[i:i+k]
            ngrams.append(igram)
        
    return ngrams
def findAllNgrams(filename):
    ids, sequences = extractSequences("preprocessed1.fasta")
    ngrams = []
    for sequence in sequences:
        ngrams += findNgrams(sequence)
    return ngrams

In [3]:
def findAllNgrams(filename):
    ids, sequences = extractSequences("preprocessed1.fasta")
    ngrams = []
    for sequence in sequences:
        ngrams += findNgrams(sequence)
    return ngrams

In [ ]:
findAllNgrams("preprocessed1.fasta")

In [20]:
findNgrams("abcde")

['ab', 'bc', 'cd', 'de', 'abc', 'bcd', 'cde', 'abcd', 'bcde']

In [ ]:
ids, seqs = extractSequences("preprocessed1.fasta")